In [1]:
from pprint import pprint
from nltk.tag import hmm
from sklearn.externals import joblib
from nltk.tag.hmm  import HiddenMarkovModelTagger, HiddenMarkovModelTrainer
from nltk.probability import LidstoneProbDist
from nltk.tokenize import sent_tokenize

import dill
import pickle
import os
import pandas as pd
import numpy as np
import sys

sys.path.append("lib/uuparser/barchybrid/src/")
import utils
from arc_hybrid import ArcHybridLSTM

In [2]:
MODELS_DIR = 'models/id_gsd/'
TAGGER_FILE_NAME = 'tagger.dill'
PARAMS_FILE = MODELS_DIR + "params.pickle"
PARSER_FILE="barchybrid.model"

In [3]:
with open(MODELS_DIR + TAGGER_FILE_NAME, 'rb') as f:
    hmm_tagger = dill.load(f)

In [4]:
with open(PARAMS_FILE, 'r') as paramsfp:
    words, w2i, pos, rels, cpos, langs, stored_opt, ch = pickle.load(paramsfp)
    parser = ArcHybridLSTM(words, pos, rels, cpos, langs, w2i,
                           ch, stored_opt)
    model = os.path.join(MODELS_DIR, PARSER_FILE)
    parser.Load(model)

Loading model from models/id_gsd/barchybrid.model


# PROCESS INDIVIDUAL REVIEW

contoh review:

sapi bakarnya enak banget harganya juga lumayan murah

In [8]:
FOOD_POSITIVE_ADJ = ['enak', 'banyk']
FOOD_NEGATIVE_ADJ = ['mahal']

PRICE_POSITIVE_ADJ = ['murah']
PRICE_NEGATIVE_ADJ = ['mahal']

SERVICE_POSITIVE_ADJ = ['murah']
SERVICE_NEGATIVE_ADJ = ['mahal']

AMBIENCE_POSITIVE_ADJ = ['murah']
AMBIENCE_NEGATIVE_ADJ = ['mahal']

'sentences'

['sapi bakarnya enak banget harganya juga lumayan murah']

In [ ]:
review = "sapi bakarnya enak banget harganya juga lumayan murah"
sentences = sent_tokenize(review)
display('sentences', sentences)

In [9]:
def examine_polarity(sentence):
    """
        Check negative positive word regarding food, price, service, and ambience aspect in particular sentence
    """
    pol_food = pol_price = pol_service = pol_ambience = 0
    
    tagged_sentence = ' '.join(['{0}/{1}'.format(word, tag) for word, tag in hmm_tagger.tag(review.split())])
    data = utils.read_conll_text(tagged_sentence)
    pred = parser.Predict(data)
    depedency_tree = list(pred)[0]
    
    for p in depedency_tree:
        print(p)
    print ""
    return pol_food, pol_price, pol_service, pol_ambience

In [10]:
for sentence in sentences:
    pol_food, pol_price, pol_service, pol_ambience = examine_polarity(sentence)

Reading sapi/NN bakarnya/FW enak/FW banget/FW harganya/FW juga/RB lumayan/RB murah/JJ
Time: 0.025s
0	*root*	*root*	ROOT-CPOS	ROOT-POS	_	-1	rroot	_	_
1	sapi	_	_	NN	_	0	advmod	_	_
2	bakarnya	_	_	FW	_	1	nsubj	_	_
3	enak	_	_	FW	_	2	nsubj	_	_
4	banget	_	_	FW	_	3	compound	_	_
5	harganya	_	_	FW	_	4	compound	_	_
6	juga	_	_	RB	_	7	advmod	_	_
7	lumayan	_	_	RB	_	5	compound	_	_
8	murah	_	_	JJ	_	7	compound	_	_

